In [1]:
# On recovering the second-order convergence of the lattice Boltzmann method with reaction-type source terms

# July 2021
# Grzegorz Gruszczyński, Michał Dzikowski, Łukasz Łaniewski-Wołłk
#
# <https://arxiv.org/abs/2107.03962>

from sympy import symbols, Eq, Matrix, solve, lambdify, diff
from sympy.solvers.ode.systems import dsolve_system
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
from eq_solver import block_simpler, extract_real_solution
import os

from sympy import init_printing
init_printing() 
# %matplotlib inline
init_printing(use_latex='mathjax')
# import handcalcs.render

## Macroscopic equation

\begin{align*}
\begin{cases}
\partial_t c_A + \nabla (u \cdot c_A) &= \nabla \cdot (D \nabla c_A)-k_A c_A +k_C c_C \\
\partial_t c_C + \nabla (u \cdot c_C) &= \nabla \cdot (D \nabla c_C)+k_A c_A -k_C c_C
\end{cases}
\end{align*}


The variable being solved in a LBM scheme is $\boldsymbol{\tilde{c}}$.

The implicit relation (due to 'shift' of variables) is

$$
\boldsymbol{\tilde{c}} = \boldsymbol{c} - \frac{1}{2} \boldsymbol{Q}
$$

$$
\begin{bmatrix} 
\tilde{c}_A \\
\tilde{c}_C\\
\end{bmatrix}
\quad
=
\begin{bmatrix} 
c_A \\
c_C \\
\end{bmatrix}
\quad
-\frac{1}{2}
\begin{bmatrix} 
-k_A c_A  &  k_C c_C \\
+k_A c_A  & -k_C c_C \\
\end{bmatrix}
\quad
$$

Let us find $\boldsymbol{c} = \boldsymbol{c}(\boldsymbol{\tilde{c}})$

In [2]:
str_ca = 'c_A'  # non-shifted quantity
str_cc = 'c_C'  # non-shifted quantity
str_k_a = 'k_A'  # source term intensity
str_k_c = 'k_C'  # source term intensity
str_dt = 'dt'
str_tilde_ca = r'\tilde{c}_A'  # shifted quantity
str_tilde_cc = r'\tilde{c}_C'  # shifted quantity
str_Q = 'Q'

ca = symbols(f"{str_ca}", real=True)  # non-shifted quantity
cc = symbols(f"{str_cc}", real=True)  # non-shifted quantity
k_a = symbols(f'{str_k_a}', real=True, positive=True, nonzero=True)
k_c = symbols(f'{str_k_c}', real=True, positive=True, nonzero=True)
DT = symbols(f'{str_dt}', real=True, positive=True, nonzero=True)  # DT=1 in LBM
DT =1 
tilde_ca = symbols(f'{str_tilde_ca}', real=True)
tilde_cc = symbols(f'{str_tilde_cc}', real=True)

given = [tilde_ca, tilde_cc]
unknown = [ca, cc]
Q = [- k_a * ca + k_c *cc, k_a*ca - k_c*cc]

EQs = Eq(Matrix(given), Matrix(unknown) - DT*Matrix(Q)/2)


In [3]:
solutions = solve(EQs, unknown, dict=True)
symbolic_solutions_as_matrix = Matrix([list(s.values()) for s in solutions])
symbolic_solutions_as_matrix

inputs_as_symbols = [tilde_ca, tilde_cc, k_a, k_c]
inputs_as_str = symbols([str_tilde_ca, str_tilde_cc, str_k_a, str_k_c])


In [4]:
solutions

⎡⎧     \tilde{c}_A⋅k_C + 2⋅\tilde{c}_A + \tilde{c}_C⋅k_C       \tilde{c}_A⋅k_A + \tilde{c}_C⋅k_A + 2⋅\tilde{c}_C⎫⎤
⎢⎨c_A: ─────────────────────────────────────────────────, c_C: ─────────────────────────────────────────────────⎬⎥
⎣⎩                       k_A + k_C + 2                                           k_A + k_C + 2                  ⎭⎦

In [20]:
latex(solutions)

'\\left[ \\left\\{ c_{A} : \\frac{\\tilde{c}_A k_{C} + 2 \\tilde{c}_A + \\tilde{c}_C k_{C}}{k_{A} + k_{C} + 2}, \\  c_{C} : \\frac{\\tilde{c}_A k_{A} + \\tilde{c}_C k_{A} + 2 \\tilde{c}_C}{k_{A} + k_{C} + 2}\\right\\}\\right]'

In [5]:
Q

[-c_A⋅k_A + c_C⋅k_C, c_A⋅k_A - c_C⋅k_C]

In [6]:
print("//=== THIS IS AUTOMATICALLY GENERATED CODE ===")
# print(f'const real_t {str_dt} = 1.;')
print(f'real_t {str_ca}; real_t {str_cc};')

# inputs_as_symbols.append(ca)
# inputs_as_str.append(str_ca)

block_simpler([str_ca], [symbolic_solutions_as_matrix.subs(dict(zip(inputs_as_symbols, inputs_as_str)))[0]])  
block_simpler([str_cc], [symbolic_solutions_as_matrix.subs(dict(zip(inputs_as_symbols, inputs_as_str)))[1]])

//=== THIS IS AUTOMATICALLY GENERATED CODE ===
real_t c_A; real_t c_C;
c_A = (\tilde{c}_A*k_C + 2*\tilde{c}_A + \tilde{c}_C*k_C)/(k_A + k_C + 2) ; // 8
c_C = (\tilde{c}_A*k_A + \tilde{c}_C*k_A + 2*\tilde{c}_C)/(k_A + k_C + 2) ; // 8


In [7]:
print(f'\nreal_t {str_Q}a; real_t {str_Q}b;')
block_simpler(["Qa"], [Q[0].subs(dict(zip(inputs_as_symbols, inputs_as_str)))])
block_simpler(["Qc"], [Q[1].subs(dict(zip(inputs_as_symbols, inputs_as_str)))])


real_t Qa; real_t Qb;
Qa = -c_A*k_A + c_C*k_C ; // 3
Qc = c_A*k_A - c_C*k_C ; // 3


## Let us solve a system of ODE in sympy

In [16]:
# redefine 'concentrations' as sympy function
ca = symbols(f"{str_ca}", real=True, cls=Function)  # non-shifted quantity
cc = symbols(f"{str_cc}", real=True, cls=Function)  # non-shifted quantity
k_a = symbols(f'{str_k_a}', real=True, positive=True, nonzero=True)
k_c = symbols(f'{str_k_c}', real=True, positive=True, nonzero=True)
t = symbols('t', real=True, positive=True)



# IC={ca(0):40, cc(0): 10} # initial condition
k_a_num = 1; k_c_num = 3   # specify value of the constant

# IC={ca(0): symbols(f'{str_ca}_0', real=True, positive=True, nonzero=True),
#     cc(0): symbols(f'{str_cc}_0', real=True, positive=True, nonzero=True)} # initial condition
# https://stackoverflow.com/questions/71708863/derivative-or-diff-in-sympy-ode
odesys = [diff(ca(t),t,1)  -k_a * ca(t)+k_c*cc(t),
          diff(cc(t),t,1)  +k_a * ca(t)-k_c*cc(t)]
odesys

⎡                           d                                     d         ⎤
⎢-k_A⋅c_A(t) + k_C⋅c_C(t) + ──(c_A(t)), k_A⋅c_A(t) - k_C⋅c_C(t) + ──(c_C(t))⎥
⎣                           dt                                    dt        ⎦

'\\left[ - k_{A} c_{A}{\\left(t \\right)} + k_{C} c_{C}{\\left(t \\right)} + \\frac{d}{d t} c_{A}{\\left(t \\right)}, \\  k_{A} c_{A}{\\left(t \\right)} - k_{C} c_{C}{\\left(t \\right)} + \\frac{d}{d t} c_{C}{\\left(t \\right)}\\right]'

In [17]:
general_solution=dsolve_system(odesys)[0];
general_solution

⎡         C₁⋅k_C       t⋅(k_A + k_C)                    t⋅(k_A + k_C)⎤
⎢c_A(t) = ────── - C₂⋅ℯ             , c_C(t) = C₁ + C₂⋅ℯ             ⎥
⎣          k_A                                                       ⎦

In [22]:
latex(general_solution)

'\\left[ c_{A}{\\left(t \\right)} = \\frac{C_{1} k_{C}}{k_{A}} - C_{2} e^{t \\left(k_{A} + k_{C}\\right)}, \\  c_{C}{\\left(t \\right)} = C_{1} + C_{2} e^{t \\left(k_{A} + k_{C}\\right)}\\right]'

In [18]:
particular_solution = dsolve_system(odesys, ics=IC)[0]; particular_solution

⎡         c_A_0⋅k_C   c_C_0⋅k_C   ⎛c_A_0⋅k_A   c_C_0⋅k_C⎞  t⋅(k_A + k_C)           c_A_0⋅k_A   c_C_0⋅k_A   ⎛c_A_0⋅k_A   c_C_0⋅k_C⎞  t⋅(k_A + k_C)⎤
⎢c_A(t) = ───────── + ───────── + ⎜───────── - ─────────⎟⋅ℯ             , c_C(t) = ───────── + ───────── - ⎜───────── - ─────────⎟⋅ℯ             ⎥
⎣         k_A + k_C   k_A + k_C   ⎝k_A + k_C   k_A + k_C⎠                          k_A + k_C   k_A + k_C   ⎝k_A + k_C   k_A + k_C⎠               ⎦

In [11]:
fun_result = [r.subs([(k_a, k_a_num),(k_c, k_c_num)]) for r in particular_solution]; fun_result

⎡         3⋅k_A_0   ⎛k_A_0   15⎞  4⋅t   15           k_A_0   ⎛k_A_0   15⎞  4⋅t   5⎤
⎢c_A(t) = ─────── + ⎜───── - ──⎟⋅ℯ    + ──, c_C(t) = ───── - ⎜───── - ──⎟⋅ℯ    + ─⎥
⎣            4      ⎝  4     2 ⎠        2              4     ⎝  4     2 ⎠        2⎦

In [12]:
# evaluate at a given time spot
t_spot = 5
v = [v.evalf(subs={t:t_spot}) for v in fun_result]
v

[c_A(5) = 121291299.602448⋅k_A_0 - 3638738958.07343, c_C(5) = 3638738968.07343 - 121291298.602448⋅k_A_0]